# Importing Libraries

In [ ]:
from __future__ import print_function
import torch
import torch.optim
from functions.models.optim import *

from skimage.measure import compare_psnr
from functions.models import *
from copy import deepcopy
from functions.utils.global_parameters import *
from functions.utils.common_utils import torch_to_np
from IPython.core.debugger import set_trace

import matplotlib.pyplot as plt
import numpy as np
import cv2

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark =True
#dtype = torch.cuda.FloatTensor
dtype = torch.FloatTensor

In [5]:
def dip(img_np, arch = 'default', LR = 0.01, num_iter = 1000, reg_noise_std = 1.0/30,exp_weight = 0.99, INPUT = 'noise', save = False, save_path = '', plot = True, input_depth = None, name = None, loss_fn = "MSE", OPTIMIZER = "adam", pad = 'zero',  OPT_OVER = 'net' ):
    
    glparam = global_parameters()
    glparam.set_params(save, plot, reg_noise_std, exp_weight)
    glparam.load_images(img_np)
    glparam.img_torch = glparam.img_torch.type(dtype)
    
    if arch == 'simple':
        if input_depth == None:
            input_depth = 3 
        glparam.net = get_net(input_depth,'skip', pad,
                skip_n33d=16, 
                skip_n33u=16, 
                skip_n11=0, 
                num_scales=3,
                upsample_mode='bilinear').type(dtype)
    else:
        assert False

    net_input = get_noise(input_depth, INPUT, (glparam.img_np.shape[1], glparam.img_np.shape[2])).type(dtype).detach()   
    glparam.net_input_saved = net_input.detach().clone()
    glparam.noise = net_input.detach().clone()
    
    # Compute number of parameters
    param_numbers  = sum([np.prod(list(p.size())) for p in glparam.net.parameters()]) 
    print ('\n Number of params: %d' % param_numbers)

    # Loss function
    if loss_fn == 'MSE':
        criterion = torch.nn.MSELoss().type(dtype)
    if loss_fn == 'KLDiv':
        criterion = torch.nn.KLDivLoss().type(dtype)
        
    if save == True:
        f= open("{}/Stats.txt".format(save_path),"w+")
        f.write("{:>11}{:>12}{:>12}\n".format('Iterations','Total_Loss','PSNR'))
        save_net_details(save_path, arch, param_numbers, pad, OPT_OVER, OPTIMIZER, input_depth,
                 loss_fn = loss_fn, LR = LR, num_iter = num_iter, exp_weight = glparam.exp,
                 reg_noise_std = reg_noise_std, INPUT = 'INPUT', net = glparam.net)
                
    def closure(iter_value):
        show_every = 100
        figsize = 4
        
        ## Initialiaze/ Update variables
        if glparam.noise_std > 0.0:
            net_input = glparam.net_input_saved + (glparam.noise.normal_() * glparam.noise_std)
        net_input = torch.tensor(net_input, dtype=torch.float32, requires_grad=True)
        out = glparam.net(net_input)#, requires_grad=True)

        ## Exponential Smoothing
        if glparam.out_avg is None:
            glparam.out_avg = out.detach()
        else:
            glparam.out_avg = glparam.out_avg * glparam.exp + out.detach() * (1 - glparam.exp)
        
        ## Calculate loss
        total_loss = criterion(out, glparam.img_torch)
        set_trace()
        total_loss.backward()
        set_trace()
        
        glparam.psnr_noisy = compare_psnr(glparam.img_np, out.detach().cpu().numpy()[0]).astype(np.float32)
            
        print ('DIP Iteration {:>11}   Loss {:>11.7f}   PSNR_noisy: {:>5.4f}'.format(
            iter_value, total_loss.item(), glparam.psnr_noisy), end='\r')
        
        ## Backtracking   
        if (glparam.psnr_noisy_last - glparam.psnr_noisy) > 5.0:
            glparam.interrupts = glparam.interrupts + 1
            print('\n Falling back to previous checkpoint.')
            glparam.net.load_state_dict(glparam.last_net.state_dict())
            glparam.optimizer.load_state_dict(glparam.optimizer_last.state_dict())
            
            if glparam.interrupts > 3:
                glparam.psnr_noisy_last = glparam.psnr_noisy
                
            if OPTIMIZER == "adam":     
                for j in range(iter_value % show_every - 1):                
                    glparam.optimizer.zero_grad()
                    closure(iter_value - (iter_value % show_every) + j + 1)
                    glparam.optimizer.step()
                glparam.optimizer.zero_grad()
                closure(iter_value)          
                print('\n Return back to the original')                        
                return total_loss 
            
            if OPTIMIZER == "EntropySGD":
                for j in range(iter_value % show_every - 1):
                    glparam.optimizer.zero_grad()
                    glparam.optimizer.step(iter_value - (iter_value % show_every) + j + 1, closure, glparam.net, criterion)
                glparam.optimizer.zero_grad()
                closure(iter_value)   
                print('\n Return back to the original')                        
                return total_loss                      
            
        if (iter_value % show_every) == 0: 
            glparam.last_net = deepcopy(glparam.net)
            glparam.psnr_noisy_last = glparam.psnr_noisy
            glparam.optimizer_last = deepcopy(glparam.optimizer)
            
            if glparam.interrupts > 3 :
                print("\n Error, was not able to converge after reset")
            glparam.interrupts = 0
            
            if glparam.PLOT:
                fig=plt.figure(figsize=(16, 16))
                fig.add_subplot(1, 3, 1)
                plt.imshow(np.clip(torch_to_np(out), 0, 1).transpose(1, 2, 0))
                plt.title('Output')
                fig.add_subplot(1, 3, 2)
                plt.imshow(np.clip(torch_to_np(glparam.out_avg), 0, 1).transpose(1, 2, 0))
                plt.title('Averaged Output')
                fig.add_subplot(1, 3, 3)
                plt.title('Original/Target')
                plt.imshow(glparam.img_np.transpose(1, 2, 0))
                plt.show()
                
            if glparam.save:
                f = open("{}/Stats.txt".format(save_path),"a")
                f.write("{:>11}{:>12.8f}{:>12.8f}\n".format(iter_value, total_loss.item(), glparam.psnr_noisy))
                plt.imsave("{}/it_{}.png".format(save_path,iter_value),
                       np.clip(torch_to_np(glparam.out_avg), 0, 1).transpose(1,2,0), format="png")
                
        return total_loss
        
    ### Optimize
    glparam.net.train()
    p = get_params(OPT_OVER, glparam.net, net_input)
    
    if OPTIMIZER == "adam":
        glparam.optimizer = torch.optim.Adam(p, lr = LR)
        for j in range(num_iter):
            glparam.optimizer.zero_grad()
            closure(j)
            glparam.optimizer.step()            
    if OPTIMIZER == "EntropySGD":
        glparam.optimizer = EntropySGD(p,config=dict(lr = LR))
        for j in range(num_iter):
            glparam.optimizer.zero_grad()
            glparam.optimizer.step(j, closure, glparam.net, criterion)    
    print('\n')       
    
    out = glparam.net(net_input)
    glparam.out_avg = glparam.out_avg * glparam.exp + out.detach() * (1 - glparam.exp)
    return glparam.out_avg

In [ ]:
import cv2
img = cv2.imread('data/goldfish.jpg')[..., ::-1]
img_np = np.array(img)
_, dip(img_np, arch = 'simple')


 Number of params: 20355


C:\ProgramData\Anaconda3\lib\site-packages\torch\nn\modules\upsampling.py:122: UserWarning: nn.Upsampling is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.Upsampling is deprecated. Use nn.functional.interpolate instead.")
C:\ProgramData\Anaconda3\lib\site-packages\torch\nn\functional.py:1961: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


> <ipython-input-5-3409fdc6e08d>(60)closure()
     58         total_loss = criterion(out, glparam.img_torch)
     59         set_trace()
---> 60         total_loss.backward()
     61         set_trace()
     62 

ipdb> c
> <ipython-input-5-3409fdc6e08d>(63)closure()
     61         set_trace()
     62 
---> 63         glparam.psnr_noisy = compare_psnr(glparam.img_np, out.detach().cpu().numpy()[0]).astype(np.float32)
     64 
     65         print ('DIP Iteration {:>11}   Loss {:>11.7f}   PSNR_noisy: {:>5.4f}'.format(

ipdb> net_input.grad.data.numpy()
array([[[[ 1.59423067e-07,  1.75658705e-08, -1.38406193e-07, ...,
           2.26717830e-07, -2.60561023e-07,  1.25579938e-08],
         [-7.58807559e-08, -1.14300065e-07, -1.25033779e-07, ...,
           1.11047669e-07, -3.28132899e-09,  4.95357085e-07],
         [ 1.25042391e-07, -3.29774025e-07,  1.30315314e-07, ...,
           7.55471703e-08, -1.80026696e-08, -3.64283778e-07],
         ...,
         [-1.82531124e-07,  1.99314542e-07, 